In [ ]:
import hilbert
import conversions

#includes for genetic
import population
import psnr
import crossover


In [ ]:
def genPoint(offsetX,offsetY,x,y):
    for i in range(512*512):
        x[i]=(x[i]+offsetX)%512
    for i in range(512*512):
        y[i]=(y[i]+offsetY)%512
    return x,y

In [ ]:
import cv2

# image_name = input("Enter Cover Image Name: ")
image_name = "lena_gray.bmp"
image = cv2.imread(image_name, 0)


Creating Secret Data from the Secret Image

In [ ]:
secret_image = "lena_secret.bmp"
secretImage = cv2.imread(image_name, 0)

b = bytearray(secretImage)

secretData = ""

for i in b:
    secretData = secretData + str(conversions.int2binary(i))

secretFile = open("secret.txt", 'w')
secretFile.write(secretData)



In [ ]:

def encodeImage(image,x,y,sbDir,sbPole):
    secretDataIndex=0
    if sbDir==1:
        if sbPole==1:
            for i in range(512*512):
                r = conversions.int2binary(image[x[i]][y[i]])
                image[x[i]][y[i]] = int(
                    r[:-3] + str(1-int(secretData[secretDataIndex+2])) + str(1-int(secretData[secretDataIndex+1])) + str(1-int(secretData[secretDataIndex])), 2)
                secretDataIndex = secretDataIndex+3
        else:
            for i in range(512*512):
                r = conversions.int2binary(image[x[i]][y[i]])
                image[x[i]][y[i]] = int(
                    r[:-3] + str(secretData[secretDataIndex+2]) + str(secretData[secretDataIndex+1]) + str(secretData[secretDataIndex]), 2)
                secretDataIndex = secretDataIndex+3
    else:
        if sbPole==1:
            for i in range(512*512):
                r = conversions.int2binary(image[x[i]][y[i]])
                image[x[i]][y[i]] = int(
                    r[:-3] + str(1-int(secretData[secretDataIndex])) + str(1-int(secretData[secretDataIndex+1])) + str(1-int(secretData[secretDataIndex+2])), 2)
                secretDataIndex = secretDataIndex+3
        else:
            for i in range(512*512):
                r = conversions.int2binary(image[x[i]][y[i]])
                image[x[i]][y[i]] = int(
                    r[:-3] + str(secretData[secretDataIndex]) + str(secretData[secretDataIndex+1]) + str(secretData[secretDataIndex+2]), 2)
                secretDataIndex = secretDataIndex+3
   
    file_name = "encoded_image.bmp"
    cv2.imwrite(file_name, image)

    return cv2.imread(file_name, 0)

In [ ]:
def encodeFunction(strr, image):
    sbDir = strr[0]
    sbPole = strr[1]
    yValue = strr[2:11]
    xValue = strr[11:20]

    xOff = int(xValue,2)
    yOff = int(yValue,2)
    x, y = hilbert.genHilbert(512, 512)
    x,y=genPoint(xOff,yOff,x,y)
    stegoImage = encodeImage(image, x,y,int(sbDir),int(sbPole))

    return stegoImage

In [ ]:
def decodeImage(image,x,y,sbDir,sbPole):
    secretData = ""
    cnt=0
    if sbDir==1:
        if sbPole==1:
            for i in range(512*512):
                r = conversions.int2binary(image[x[i]][y[i]])
                secretData+=str(1-int(r[7])) + str(1-int(r[6]))+str(1-int(r[5]))
        else:
            for i in range(512*512):
                r = conversions.int2binary(image[x[i]][y[i]])
                secretData+= str(r[7]) + str(r[6])+str(r[5])
    else:
        if sbPole==1:
            for i in range(512*512):
                r = conversions.int2binary(image[x[i]][y[i]])
                secretData+=str(1-int(r[5])) + str(1-int(r[6]))+str(1-int(r[7]))
        else:
            for i in range(512*512):
                r = conversions.int2binary(image[x[i]][y[i]])
                secretData+=str(r[5])+str(r[6])+str(r[7])
    secretFile = open("decodedData.txt", 'w')
    secretFile.write(secretData)

In [ ]:
def decodeFunction(strr, image):
    sbDir = strr[0]
    sbPole = strr[1]
    yValue = strr[2:11]
    xValue = strr[11:20]

    xOff = int(xValue,2)
    yOff = int(yValue,2)
    x, y = hilbert.genHilbert(512, 512)
    x,y=genPoint(xOff,yOff,x,y)
    decodeImage(image,x,y,sbDir,sbPole)
    

Generating Initial Population For Genetic

In [ ]:
# it = 4

# mainPopulation = population.genPopulation(4)

# print(mainPopulation)
# fitnessArray = []*it


# for i in range(it):
#     print("Parent - ", i)
image = cv2.imread(image_name, 0)
stegoImage = encodeFunction("00000000000000000000",image)
originalImage = cv2.imread(image_name, 0)
print(psnr.computeFitness(512, originalImage, stegoImage))
decodeFunction("00000000000000000000",stegoImage)

# print(fitnessArray)

